In [52]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
from copy import deepcopy

from camera_conventions.parser import parse_ini_convention
Blender = parse_ini_convention('../conventions/Blender.ini')
OpenGL = parse_ini_convention('../conventions/OpenGL.ini')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from camera_conventions.geometry import Geometry

arr = np.array([    
    0.0, 0.2, 0.5,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.7,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7,
    0.0, 0.2, 0.5,      
    -0.2, -0.1, 0.3,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.3, 
    0.2, -0.1, 0.7,
    -0.2, -0.1, 0.7,
    -0.2, -0.1, 0.3, 
    0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7]).reshape(-1, 3)
colors = np.array([
    1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0,
                  0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,
                  0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0,
                  1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0
]).reshape(-1, 3)
colors = (colors * 255).astype(np.uint8)
arr[:, 2] -= 0.5
verts = arr
faces = np.arange(0, verts.shape[0]).reshape(-1, 3)
geom = Geometry(convention=OpenGL, verts=verts, faces=faces, vertex_colors=colors)
geom.to_json('../assets/geometries/pyramid.json')
geom.convert(Blender).to_json('../assets/geometries/pyramid_blender.json')

In [43]:
from camera_conventions.camera import Camera

"""
(((-0.7071067690849304, 3.0908619663705394e-08, 0.7071067690849304, 1.0),
        (0.7071067690849304, 3.0908619663705394e-08, 0.7071067690849304, 1.0),
        (0.0, 1.0, -4.371138828673793e-08, 0.0),
        (0.0, 0.0, 0.0, 1.0)))
"""
bTestCam = Camera(
    convention=Blender,
    fhat=50.0 / 36.0,
    aspect_ratio=1.5,
    T=np.array(
        [
            [-np.sqrt(2) / 2, 0, np.sqrt(2) / 2, 1.0],
            [np.sqrt(2) / 2, 0, np.sqrt(2) / 2, 1.0],
            [0, 1.0, 0, 0],
            [0, 0, 0, 1.0],
        ]
    ),
)
bTestCam.to_json('../assets/cameras/testCam_blender.json')
bTestCam.T

array([[-0.70710678,  0.        ,  0.70710678,  1.        ],
       [ 0.70710678,  0.        ,  0.70710678,  1.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [44]:
Rt, C = bTestCam.T[:3, :3], bTestCam.T[:3, 3]
blender_to_opengl = np.linalg.inv(Blender.world_transformation_matrix)
Rt_opengl = blender_to_opengl @ Rt
C_opengl = blender_to_opengl @ C
opengl_T = np.eye(4)
opengl_T[:3, :3] = Rt_opengl
opengl_T[:3, 3] = C_opengl
opengl_T = np.linalg.inv(opengl_T)
opengl_T

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [45]:
opengl_to_blender = np.eye(4)
opengl_to_blender[:3, :3] = np.linalg.inv(blender_to_opengl)
np.linalg.inv(bTestCam.T) @ opengl_to_blender

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [46]:
bTestCam.convert(OpenGL).T

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [47]:
np.testing.assert_allclose(bTestCam.convert(OpenGL).convert(Blender).T, bTestCam.T)

In [48]:
bTestCam.convert(OpenGL).to_json('../assets/cameras/testCam.json')

In [54]:
AliceVision = parse_ini_convention('../conventions/alicevision.ini')
with open('../assets/cameras/robCam.txt') as f:
    for i in range(3):
        f.readline()
    T = np.array(list(map(float, f.read().split()))).reshape(4, 4)
    T[:3, 3] /= 100
alicecam = Camera(
    fhat=  20063 / 6144,
    convention = AliceVision,
    aspect_ratio=3240 / 6144,
    T = T
)
alicecam.convert(Blender).to_json('../assets/cameras/robCam_blender.json')
alicecam

Camera(fhat=3.2654622395833335, T=array([[ 0.44548106, -0.02055738, -0.89505532, -1.47978022],
       [-0.04920658,  0.99766302, -0.04740482, -0.14007912],
       [ 0.89393811,  0.06516056,  0.44342842,  0.70026039],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), aspect_ratio=0.52734375, convention=AliceVision)

In [64]:
from camera_conventions.camera import parse_alicevision_camera
parse_alicevision_camera("/nas/RobFacialPerformace/00/cameras/camera0.txt", AliceVision)

Camera(fhat=3.2655901054237084, T=array([[ 0.44548106, -0.02055738, -0.89505532, -1.47978022],
       [-0.04920658,  0.99766302, -0.04740482, -0.14007912],
       [ 0.89393811,  0.06516056,  0.44342842,  0.70026039],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), aspect_ratio=0.52734375, convention=AliceVision)